# Kickstarter 

### Loading relevent libraries and dataset

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
from nltk import word_tokenize

In [2]:
train = pd.read_csv('train.csv')
train.head(3)

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,False,US,USD,1241333999,1241334017,1240600507,1240602723,3,1
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,False,US,USD,1242429000,1242432018,1240960224,1240975592,2,0
2,kkst183622197,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,mr-squiggles,False,US,USD,1243027560,1243027818,1242163613,1242164398,0,0


In [3]:
test = pd.read_csv('test.csv')
test.head(3)

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at
0,kkst917493670,Bràthair.,"My first film, of many to come. Trying to purs...",7000.0,brathair,False,US,USD,1449619185,1449619185,1446002581,1446159585
1,kkst1664901914,THE SCREENWRITER,A young man that has earned his master's in sc...,35000.0,the-screenwriter,False,US,USD,1453435620,1453435620,1450297323,1450411620
2,kkst925125077,The Hornets Nest the Fairmont Heights Story,Film about a high school constructed for negro...,49500.0,the-hornets-nest-the-fairmont-heights-story,False,US,USD,1451780700,1451780700,1448581356,1448672128


### Analyzing datasets

In [4]:
print('train length:{0}'.format(len(train)))
print('test length:{0}'.format(len(test)))

train length:108129
test length:63465


In [5]:
print('train length:{0}\n'.format(train.dtypes))
print('test length:{0}'.format(test.dtypes))

train length:project_id                object
name                      object
desc                      object
goal                     float64
keywords                  object
disable_communication       bool
country                   object
currency                  object
deadline                   int64
state_changed_at           int64
created_at                 int64
launched_at                int64
backers_count              int64
final_status               int64
dtype: object

test length:project_id                object
name                      object
desc                      object
goal                     float64
keywords                  object
disable_communication       bool
country                   object
currency                  object
deadline                   int64
state_changed_at           int64
created_at                 int64
launched_at                int64
dtype: object


In [6]:
X = train.copy()
y = test.copy()

In [7]:
X = pd.concat([X,pd.DataFrame(columns=['dl_y','dl_m','dl_d','sc_y','sc_m','sc_d','ca_y','ca_m','ca_d','la_y','la_m','la_d'])])
X.columns

Index(['backers_count', 'ca_d', 'ca_m', 'ca_y', 'country', 'created_at',
       'currency', 'deadline', 'desc', 'disable_communication', 'dl_d', 'dl_m',
       'dl_y', 'final_status', 'goal', 'keywords', 'la_d', 'la_m', 'la_y',
       'launched_at', 'name', 'project_id', 'sc_d', 'sc_m', 'sc_y',
       'state_changed_at'],
      dtype='object')

In [8]:
X['dl_y'] = 0
X['dl_m'] = 0
X['dl_d'] = 0
X['sc_y'] = 0
X['sc_m'] = 0
X['sc_d'] = 0
X['ca_y'] = 0
X['ca_m'] = 0
X['ca_d'] = 0
X['la_y'] = 0
X['la_m'] = 0
X['la_d'] = 0

In [9]:
for i, row in X.iterrows():
    date_d = datetime.fromtimestamp(row['deadline']).strftime('%Y-%m-%d')
    date_s = datetime.fromtimestamp(row['state_changed_at']).strftime('%Y-%m-%d')
    date_c = datetime.fromtimestamp(row['created_at']).strftime('%Y-%m-%d')
    date_l = datetime.fromtimestamp(row['launched_at']).strftime('%Y-%m-%d')
    X.at[i,'dl_y'] = date_d[:4]
    X.at[i,'dl_m'] = date_d[5:7]
    X.at[i,'dl_d'] = date_d[8:]
    X.at[i,'sc_y'] = date_s[:4]
    X.at[i,'sc_m'] = date_s[5:7]
    X.at[i,'sc_d'] = date_s[8:]
    X.at[i,'ca_y'] = date_c[:4]
    X.at[i,'ca_m'] = date_c[5:7]
    X.at[i,'ca_d'] = date_c[8:]
    X.at[i,'la_y'] = date_l[:4]
    X.at[i,'la_m'] = date_l[5:7]
    X.at[i,'la_d'] = date_l[8:]

In [10]:
X.drop(['deadline','state_changed_at','created_at','launched_at'], axis=1, inplace=True)
X.columns

Index(['backers_count', 'ca_d', 'ca_m', 'ca_y', 'country', 'currency', 'desc',
       'disable_communication', 'dl_d', 'dl_m', 'dl_y', 'final_status', 'goal',
       'keywords', 'la_d', 'la_m', 'la_y', 'name', 'project_id', 'sc_d',
       'sc_m', 'sc_y'],
      dtype='object')

In [11]:
X.iloc[:,8:20].head(10)

,dl_d,dl_m,dl_y,final_status,goal,keywords,la_d,la_m,la_y,name,project_id,sc_d
0,3,5,2009,1.0,20.0,drawing-for-dollars,25,4,2009,drawing for dollars,kkst1451568084,3
1,16,5,2009,0.0,300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,29,4,2009,Sponsor Dereck Blackburn (Lostwars) Artist in ...,kkst1474482071,16
2,23,5,2009,0.0,30.0,mr-squiggles,13,5,2009,Mr. Squiggles,kkst183622197,23
3,29,5,2009,1.0,500.0,help-me-write-my-second-novel,29,4,2009,Help me write my second novel.,kkst597742710,29
4,31,5,2009,0.0,2000.0,support-casting-my-sculpture-in-bronze,1,5,2009,Support casting my sculpture in bronze,kkst1913131122,31
5,1,6,2009,0.0,700.0,daily-digest,5,5,2009,daily digest,kkst1085176748,1
6,1,6,2009,0.0,250.0,igoozex-free-iphone-app,8,5,2009,iGoozex - Free iPhone app,kkst1468954715,1
7,4,6,2009,1.0,1000.0,drive-a-faster-car-20,5,5,2009,Drive A Faster Car 2.0,kkst194050612,4
8,6,6,2009,0.0,5000.0,lostles-at-tinys-giant,5,5,2009,"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""...",kkst708883590,6
9,14,6,2009,0.0,3500.0,choose-your-own-adventure-a-robot-painting-series,14,5,2009,Choose Your Own Adventure - A Robot Painting S...,kkst890976740,14


In [12]:
X['duration'] = 0
for i, row in X.iterrows():
    dur = date(X.iloc[i]['dl_y'],X.iloc[i]['dl_m'],X.iloc[i]['dl_d']) - date(X.iloc[i]['ca_y'],X.iloc[i]['ca_m'],X.iloc[i]['ca_d'])
    X.at[i,'duration'] = str(dur).split(' ')[0]

In [13]:
X.drop(['dl_d','sc_d','ca_d','la_d'], axis=1, inplace=True)

In [14]:
X['disable_communication'].replace([False,True],[0,1], inplace=True)

In [15]:
country = pd.get_dummies(X['country'],prefix='cntry')
X = pd.concat([X, country], axis=1)

In [16]:
X.drop('cntry_DE', axis=1, inplace=True)

In [17]:
X.drop('country', axis=1, inplace=True)

In [18]:
currency = pd.get_dummies(X['currency'],prefix='curr')
X = pd.concat([X, currency], axis=1)

In [19]:
X.drop('currency', axis=1, inplace=True)

In [20]:
X.dropna(axis=0, how='any', inplace=True)
print(X.isnull().sum())
print(len(X))

backers_count            0
ca_m                     0
ca_y                     0
desc                     0
disable_communication    0
dl_m                     0
dl_y                     0
final_status             0
goal                     0
keywords                 0
la_m                     0
la_y                     0
name                     0
project_id               0
sc_m                     0
sc_y                     0
duration                 0
cntry_AU                 0
cntry_CA                 0
cntry_DK                 0
cntry_GB                 0
cntry_IE                 0
cntry_NL                 0
cntry_NO                 0
cntry_NZ                 0
cntry_SE                 0
cntry_US                 0
curr_AUD                 0
curr_CAD                 0
curr_DKK                 0
curr_EUR                 0
curr_GBP                 0
curr_NOK                 0
curr_NZD                 0
curr_SEK                 0
curr_USD                 0
dtype: int64
108121


In [21]:
tokens = list()
for i, row in X.iterrows():
    toks = row['keywords'].split('-')
    tokens.append(toks)

In [22]:
stop_clean = list()
for j in tokens:
    temp = list()
    for i in j :
        if i.lower() not in stopwords.words('english'):
            temp.append(i.lower())
    stop_clean.append(temp)

In [23]:
lemm_toks = list()
lemm = WordNetLemmatizer()
for i in stop_clean:
    temp = list()
    for j in i:
        word = lemm.lemmatize(j,'a')
        word = lemm.lemmatize(word,'s')
        word = lemm.lemmatize(word,'r')
        word = lemm.lemmatize(word,'n')
        word = lemm.lemmatize(word,'v')
        temp.append(word)
    lemm_toks.append(temp)

In [24]:
master_list = list()
for i in lemm_toks:
    for j in i:
        master_list.append(j)

In [25]:
master_filter = [i for i in master_list if len(i)>2]

In [26]:
word_cnt = dict()
for val in master_filter:
    if val in word_cnt:
        word_cnt[val] += 1
    else:
        word_cnt[val] = 1

In [27]:
word_cnt_filter = {k:v for k,v in word_cnt.items() if v >= 5}    

In [28]:
coupons = list()
for i, row in X.iterrows():
    coup = word_tokenize(row['desc'])
    coupons.append(coup)

In [29]:
coupons_lower = list()
for i in coupons:
    temp = list()
    for j in i:
        if len(j) > 2:
            temp.append(j.lower())
    coupons_lower.append(temp)

In [30]:
stop_coupons = list()
for j in coupons_lower:
    temp = list()
    for i in j :
        if i.lower() not in stopwords.words('english'):
            temp.append(i)
    stop_coupons.append(temp)

In [31]:
lemm_coup = list()
lemm = WordNetLemmatizer()
for i in stop_coupons:
    temp = list()
    for j in i:
        word = lemm.lemmatize(j,'a')
        word = lemm.lemmatize(word,'s')
        word = lemm.lemmatize(word,'r')
        word = lemm.lemmatize(word,'n')
        word = lemm.lemmatize(word,'v')
        temp.append(word)
    lemm_coup.append(temp)

In [32]:
master_coup = list()
for i in lemm_coup:
    for j in i:
        master_coup.append(j)

In [33]:
word_coup = dict()
for val in master_coup:
    if val in word_coup:
        word_coup[val] += 1
    else:
        word_coup[val] = 1

In [34]:
word_cnt_coup = {k:v for k,v in word_coup.items() if v >= 5} 

In [35]:
filter_keys = list()
for i, val in enumerate(lemm_toks):
    temp = list()
    coupo = lemm_coup[i]
    for j in val:
        if j in coupo:
            temp.append(j)
    filter_keys.append(temp)           

In [36]:
master_keys_list = [j for i in filter_keys for j in i]

In [37]:
for i, val in enumerate(filter_keys):
    if len(val) == 0:
        for j in lemm_coup[i]:
            if j in master_keys_list:
                filter_keys[i].append(j)

In [38]:
for i, val in enumerate(filter_keys):
    if len(val) == 0:
        for j in lemm_toks[i]:
            if j in master_keys_list:
                filter_keys[i].append(j)

In [39]:
master_words = [j for i in filter_keys for j in i]

In [40]:
master_words_count = dict()
for i in master_words:
    if i in master_words_count:
        master_words_count[i] += 1
    else:
        master_words_count[i] = 1

In [41]:
master_words_count_filter = {k:v for k,v in master_words_count.items() if v >= 25}

In [42]:
repos = list()
for k, v in master_words_count_filter.items():
    repos.append(k)

In [43]:
keywords = list()
for i,val in enumerate(filter_keys):
    temp = list()
    for j in val:
        if j in repos:
            temp.append(j)
    keywords.append(temp)

In [49]:
selected_keys = pd.DataFrame(0, index=np.arange(0), columns=repos)

In [50]:
selected_keys

,draw,fund,dollar,project,think,fun,guy,call,help,write,...,patent,kiln,tasty,throne,selfie,soy,salad,jewellery,denim,jerky


In [ ]:
selected_keys